In [34]:
import pandas as pd

In [35]:
pd.options.display.max_columns = False

### Clean `tbl_RepsAssigned` table

The `tbl_RepsAssigned` table contains detailed records of represented cases in the immigration system, including both noncitizen (Alien) and government (INS) representation. Each row represents an instance of legal representation assigned to a case.

It is essential for identifying the presence and type of legal representation in each case.

#### Initial Data Inspection

In [36]:
reps_assigned_path = "../../data/raw/tbl_RepsAssigned.csv"

reps_assigned = pd.read_csv(
    filepath_or_buffer=reps_assigned_path, delimiter="\t", low_memory=False
)

In [37]:
reps_assigned.head()

,IDNREPSASSIGNED,IDNCASE,STRATTYLEVEL,STRATTYTYPE,PARENT_TABLE,PARENT_IDN,BASE_CITY_CODE,INS_TA_DATE_ASSIGNED,E_27_DATE,E_28_DATE,BLNPRIMEATTY
0,5945062,2047751,COURT,ALIEN,A_Tblcase,2047751.0,WAS,NaN,NaN,2004-04-06 00:00:00.000,1
1,5945065,2052183,COURT,ALIEN,A_Tblcase,2052183.0,WAS,NaN,NaN,2004-04-15 00:00:00.000,1
2,5945066,2052595,COURT,ALIEN,A_Tblcase,2052595.0,WAS,NaN,NaN,1987-07-21 00:00:00.000,1
3,5945067,2054039,COURT,ALIEN,A_Tblcase,2054039.0,WAS,NaN,NaN,1994-06-15 00:00:00.000,1
4,5945068,2056837,COURT,ALIEN,A_Tblcase,2056837.0,WAS,NaN,NaN,2003-05-15 00:00:00.000,1


In [38]:
reps_assigned.shape

(24350854, 11)

In [39]:
reps_assigned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24350854 entries, 0 to 24350853
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   IDNREPSASSIGNED       int64  
 1   IDNCASE               int64  
 2   STRATTYLEVEL          object 
 3   STRATTYTYPE           object 
 4   PARENT_TABLE          object 
 5   PARENT_IDN            float64
 6   BASE_CITY_CODE        object 
 7   INS_TA_DATE_ASSIGNED  object 
 8   E_27_DATE             object 
 9   E_28_DATE             object 
 10  BLNPRIMEATTY          int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 2.0+ GB


#### Missing Values Summary

Compute the count and percentage of missing values for each column in `tbl_RepsAssigned` to assess data completeness.  
Use `.isna().sum()` for the count, divide by the total row count for the percentage, and sort columns by missing count to highlight those requiring cleaning attention.

In [40]:
null_counts = reps_assigned.isna().sum()
percent_missing = (null_counts / len(reps_assigned)) * 100

missing_summary = pd.DataFrame(
    {"Missing Count": null_counts, "Missing %": percent_missing.round(2)}
).sort_values(by="Missing Count", ascending=False)

display(missing_summary)

,Missing Count,Missing %
E_27_DATE,23405446,96.12
E_28_DATE,12693828,52.13
INS_TA_DATE_ASSIGNED,11642124,47.81
BASE_CITY_CODE,7912377,32.49
PARENT_IDN,1,0.00
IDNREPSASSIGNED,0,0.00
IDNCASE,0,0.00
STRATTYLEVEL,0,0.00
STRATTYTYPE,0,0.00
PARENT_TABLE,0,0.00


In [41]:
reps_assigned.duplicated().sum()

np.int64(0)

#### Filtering for Alien Representation

The `reps_assigned` table contains representation records for both respondents (noncitizens) and the government (DHS/INS).  
To retain only respondent-side representation, filter rows where `STRATTYTYPE` equals `ALIEN`.  
This ensures the dataset reflects legal counsel for the individual rather than government attorneys.

In [ ]:
reps_assigned["STRATTYTYPE"].unique()

In [ ]:
reps_assigned = reps_assigned[reps_assigned["STRATTYTYPE"] == "ALIEN"].reset_index(
    drop=True
)

In [ ]:
reps_assigned.shape

#### Filtering for Juvenile Cases

Filter `tbl_RepsAssigned` to retain only records where `IDNCASE` matches values from `tbl_JuvenileHistory`.  
This subset operation replaces a full table join, reducing memory usage and improving processing speed.

In [45]:
juvenile_history = pd.read_csv(
    "../../data/cleaned/juvenile_history_cleaned.csv.gz",
    usecols=["idnCase"],
    dtype={"idnCase": "Int64"},
)

In [46]:
juvenile_case_ids = juvenile_history["idnCase"].dropna().unique()

In [47]:
juvenile_reps_assigned = reps_assigned[
    reps_assigned["IDNCASE"].isin(juvenile_case_ids)
].reset_index(drop=True)

In [48]:
print(
    f"Number of idnCase keys in tbl_JuvenileHistory table: {len(juvenile_case_ids):,}"
)
print(
    f"Number of matched juvenile rows in tbl_RepsAssigned.csv based on those keys: {juvenile_reps_assigned.shape[0]:,}"
)

Number of idnCase keys in tbl_JuvenileHistory table: 1,924,938
Number of matched juvenile rows in tbl_RepsAssigned.csv based on those keys: 5,277,401


#### Context for Choosing Between `IDNCASE` and `PARENT_IDN` for Merging

Although records have already been filtered using `IDNCASE` from `tbl_JuvenileHistory`,  
it is important to understand the relationship between `IDNCASE` and `PARENT_IDN` before any future merge with the cases table.

- `PARENT_IDN`: Identifier of the case in the parent table, which may not always be `tblCase`.
- `IDNCASE`: Foreign key referencing the case in `tblCase`.

For accurate merging, it is critical to confirm whether these two fields hold identical values for rows where the parent table is `tbl_case`.  
If discrepancies exist, merge results may be incomplete or inaccurate.

In [49]:
juvenile_reps_assigned["PARENT_TABLE"].value_counts()

PARENT_TABLE
a_tblcase          2529175
b_tblproceeding    2512689
tblappeal            89456
B_TblProceeding      60105
A_Tblcase            50325
tblAppeal            34703
A_TblCase              948
Name: count, dtype: int64

Retain only records with `PARENT_TABLE` set to `a_tblcase` to focus on attorney assignments linked to original case records rather than appeals or other parent entities.

In [50]:
juvenile_reps_assigned = juvenile_reps_assigned[
    juvenile_reps_assigned["PARENT_TABLE"].isin(["A_TblCase", "A_Tblcase", "a_tblcase"])
].reset_index(drop=True)

In [62]:
juvenile_reps_assigned["PARENT_TABLE"].value_counts()

PARENT_TABLE
a_tblcase    2529175
A_Tblcase      50325
A_TblCase        948
Name: count, dtype: int64

Verify if `IDNCASE` and `PARENT_IDN` hold identical case IDs to confirm merge key choice.

In [52]:
matches = juvenile_reps_assigned["IDNCASE"] == juvenile_reps_assigned["PARENT_IDN"]
print(matches.value_counts())

True     2580425
False         23
Name: count, dtype: int64


In [53]:
mismatched_rows = juvenile_reps_assigned.query("IDNCASE != PARENT_IDN")

In [61]:
mismatched_rows.head()

,IDNREPSASSIGNED,IDNCASE,STRATTYLEVEL,STRATTYTYPE,PARENT_TABLE,PARENT_IDN,BASE_CITY_CODE,INS_TA_DATE_ASSIGNED,E_27_DATE,E_28_DATE,BLNPRIMEATTY
28197,12086960,6163622,COURT,ALIEN,a_tblcase,3317728.0,NYC,NaN,NaN,2009-03-05 00:00:00.000,0
31780,12573607,6271339,COURT,ALIEN,a_tblcase,6271917.0,DET,NaN,NaN,2010-01-28 00:00:00.000,1
35667,13110143,6597692,COURT,ALIEN,a_tblcase,6622225.0,HON,NaN,NaN,2010-12-06 00:00:00.000,1
57232,15128637,7304902,COURT,ALIEN,a_tblcase,7258006.0,NaN,NaN,NaN,2015-11-27 00:00:00.000,1
127551,16946073,6163622,COURT,ALIEN,a_tblcase,3317728.0,NYC,NaN,NaN,2009-03-05 00:00:00.000,0


Only 23 out of ~2.5M records have mismatched values between `IDNCASE` and `PARENT_IDN`.  
Given the high match rate and consistency with other tables, `IDNCASE` will be used as the merge key.

#### Rechecking Missing Values After Juvenile Case Filtering

Since the dataset has been filtered to only include juvenile cases,  
some columns may now be entirely empty or contain significantly fewer values than in the full table.  
A recheck of missing values is required to identify columns that are now fully null and can be safely dropped without losing relevant information.

In [69]:
null_counts = juvenile_reps_assigned.isna().sum()
percent_missing = (null_counts / len(juvenile_reps_assigned)) * 100

missing_summary = pd.DataFrame(
    {"Missing Count": null_counts, "Missing %": percent_missing.round(2)}
).sort_values(by="Missing Count", ascending=False)

display(missing_summary)

,Missing Count,Missing %
E_27_DATE,2580448,100.00
BASE_CITY_CODE,2467902,95.64
INS_TA_DATE_ASSIGNED,1313248,50.89
E_28_DATE,40,0.00
IDNREPSASSIGNED,0,0.00
IDNCASE,0,0.00
STRATTYLEVEL,0,0.00
STRATTYTYPE,0,0.00
PARENT_TABLE,0,0.00
PARENT_IDN,0,0.00


Several columns are now fully null after filtering to juvenile cases, most notably `E_27_DATE` (100% missing) and `BASE_CITY_CODE` (~96% missing). 

#### Selected Features – `tbl_RepsAssigned`

- **`IDNREPSASSIGNED`** – Primary key for this table.  
- **`IDNCASE`** – Foreign key linking to `tblCase`.  
- **`STRATTYLEVEL`** – Stage or jurisdiction level where the attorney participates:  
  - `"COURT"` → Immigration Court  
  - `"BOARD"` → Board of Immigration Appeals  
- **`STRATTYTYPE`** – Identifies who the attorney or representative is representing:  
  - `ALIEN` → Represents the noncitizen in the case (indicates representation)  
  - `INS` → Represents the government (Department of Homeland Security); removed from analysis.  
- **`E_28_DATE`** – Date of submission of **Form EOIR-28** (*Notice of Entry of Appearance as Attorney or Representative Before the Immigration Court*).  

In [70]:
selected_columns = [
    "IDNREPSASSIGNED",
    "IDNCASE",
    "STRATTYLEVEL",
    "STRATTYTYPE",
    "E_28_DATE",
]

In [71]:
juvenile_reps_assigned = juvenile_reps_assigned[selected_columns]

In [ ]:
juvenile_reps_assigned.head()

,IDNREPSASSIGNED,IDNCASE,STRATTYLEVEL,STRATTYTYPE,E_28_DATE
0,5947072,2607017,COURT,ALIEN,2004-09-23 00:00:00.000
1,5947352,2848551,COURT,ALIEN,1991-10-29 00:00:00.000
2,5948254,2824744,COURT,ALIEN,1999-06-25 00:00:00.000
3,5948542,2047758,COURT,ALIEN,2004-07-15 00:00:00.000
4,5949322,3131023,COURT,ALIEN,2005-05-26 00:00:00.000


#### Specifying Column Data Types

- `STRATTYLEVEL`, `STRATTYTYPE`: categorical — stored as `category` to ensure consistency and reduce memory usage.  
- `E_28_DATE`: datetime — stored as `datetime64[ns]` for accurate temporal analysis.

In [79]:
juvenile_reps_assigned.dtypes

IDNREPSASSIGNED             int64
IDNCASE                     int64
STRATTYLEVEL               object
STRATTYTYPE                object
E_28_DATE          datetime64[ns]
dtype: object

In [81]:
juvenile_reps_assigned = juvenile_reps_assigned.astype(
    {"STRATTYLEVEL": "category", "STRATTYTYPE": "category"}
)

In [83]:
for col in juvenile_reps_assigned.select_dtypes(include="category"):
    juvenile_reps_assigned[col] = juvenile_reps_assigned[
        col
    ].cat.remove_unused_categories()

#### Datetime Format Validation

Target feature for validation and conversion:
- `E_28_DATE`

All values follow the pattern `YYYY-MM-DD 00:00:00.000` (e.g., `2025-02-04 00:00:00.000`).  
Validation will be performed by checking all non-null entries against this format to ensure compatibility before applying `pd.to_datetime(errors="coerce")`.  

During transformation, only the `YYYY-MM-DD` portion of each timestamp will be retained for analysis.

In [73]:
def find_invalid_dates(df, column):
    """
    Returns non-null rows that don’t match the `YYYY-MM-DD` pattern.
    """
    return df[
        df[column].notna()
        & ~df[column].astype(str).str.contains(r"\d{4}-\d{2}-\d{2}", regex=True)
    ][[column]]

In [74]:
invalid_E_28_DATE = find_invalid_dates(juvenile_reps_assigned, "E_28_DATE")

In [75]:
def report_invalid(name, df):
    """
    Prints the number of invalid entries and displays the DataFrame if not empty.
    """
    count = len(df)
    print(f"{name}: {count} invalid entr{'y' if count == 1 else 'ies'}")
    if count > 0:
        display(df)

In [76]:
report_invalid("E_28_DATE", invalid_E_28_DATE)

E_28_DATE: 0 invalid entries


In [78]:
juvenile_reps_assigned["E_28_DATE"] = pd.to_datetime(
    juvenile_reps_assigned["E_28_DATE"], errors="coerce"
)

In [84]:
juvenile_reps_assigned.dtypes

IDNREPSASSIGNED             int64
IDNCASE                     int64
STRATTYLEVEL             category
STRATTYTYPE              category
E_28_DATE          datetime64[ns]
dtype: object

#### Representation Identification Logic

The `tbl_RepsAssigned` table serves as the primary source for determining whether a person is represented by legal counsel.  
Representation status is inferred from the `STRATTYTYPE` column:

- **Represented** — Case has at least one matching `IDNCASE` in `tbl_RepsAssigned` with `STRATTYTYPE = "ALIEN"`, indicating the attorney is representing the noncitizen.  
- **Not Represented** — Case `IDNCASE` does not appear in `tbl_RepsAssigned` with `STRATTYTYPE = "ALIEN"`.  

This method also allows counting the number of representatives per case by aggregating matching rows for each `IDNCASE`.  

Since the dataset does not provide a direct representation flag, this classification relies on accurate and consistent data entry by EOIR in `STRATTYTYPE`.

In [85]:
total_records = juvenile_reps_assigned["IDNCASE"].count()
unique_cases = juvenile_reps_assigned["IDNCASE"].nunique()
avg_reps_per_case = total_records / unique_cases

print(f"Total representative records: {total_records}")
print(f"Unique represented cases: {unique_cases}")
print(f"Average representatives per represented case: {avg_reps_per_case:.2f}")

Total representative records: 2580448
Unique represented cases: 1031166
Average representatives per represented case: 2.50
